# Pydantic Class in OpenAI

In [2]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
class WeatherSearch(BaseModel):
    """search for weather given an Airport code"""
    airport_code: str = Field(description="The code of the airport you want to search for")

In [7]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

weather_function = convert_pydantic_to_openai_function(WeatherSearch)
weather_function

{'name': 'WeatherSearch',
 'description': 'search for weather given an Airport code',
 'parameters': {'properties': {'airport_code': {'description': 'The code of the airport you want to search for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [10]:
model = ChatOpenAI(model="gpt-4o-mini")

model.invoke("What is the weather in KHI?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"KHI"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 66, 'total_tokens': 83, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7693ae462b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-5971bc47-d1ec-40e1-b06c-6bb4d2def9cb-0', usage_metadata={'input_tokens': 66, 'output_tokens': 17, 'total_tokens': 83, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [11]:
model_forced_function = model.bind(functions=[weather_function], function_call={"name": "WeatherSearch"})

In [12]:
model_forced_function.invoke("What is the weather in KHI?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"KHI"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 76, 'total_tokens': 83, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-79ae90d7-3318-4dbc-bc76-cc551ec8711c-0', usage_metadata={'input_tokens': 76, 'output_tokens': 7, 'total_tokens': 83, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [13]:
model_forced_function.invoke("HI?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 70, 'total_tokens': 77, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-d5ab1e4a-d684-4fb6-85ea-c03d82eaa65a-0', usage_metadata={'input_tokens': 70, 'output_tokens': 7, 'total_tokens': 77, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [15]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant"),
    ("human", "{input}"),
])

model = model.bind(functions=[weather_function])

chain = prompt | model

In [16]:
chain.invoke({"input": "What is the weather in KHI?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"KHI"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 71, 'total_tokens': 88, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'function_call', 'logprobs': None}, id='run-54e61436-a68c-4912-9da3-3af11338b420-0', usage_metadata={'input_tokens': 71, 'output_tokens': 17, 'total_tokens': 88, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [17]:
chain.invoke({"input": "HI?"})

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 65, 'total_tokens': 75, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-04fc39fc-6527-48fc-9fe4-327786f8be2f-0', usage_metadata={'input_tokens': 65, 'output_tokens': 10, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [18]:
class ArtistSearch(BaseModel):
    """Call this to get the name of songs by a particular artist"""
    artist_name: str = Field(description="The name of the artist you want to search for")
    n: int = Field(default=5, ge=1, description="The number of songs to return")

In [20]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

functions

[{'name': 'WeatherSearch',
  'description': 'search for weather given an Airport code',
  'parameters': {'properties': {'airport_code': {'description': 'The code of the airport you want to search for',
     'type': 'string'}},
   'required': ['airport_code'],
   'type': 'object'}},
 {'name': 'ArtistSearch',
  'description': 'Call this to get the name of songs by a particular artist',
  'parameters': {'properties': {'artist_name': {'description': 'The name of the artist you want to search for',
     'type': 'string'},
    'n': {'default': 5,
     'description': 'The number of songs to return',
     'minimum': 1,
     'type': 'integer'}},
   'required': ['artist_name'],
   'type': 'object'}}]

In [21]:
model_with_functions = model.bind(functions=functions)

In [22]:
model_with_functions.invoke("What is the weather in KHI?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"KHI"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 128, 'total_tokens': 145, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'function_call', 'logprobs': None}, id='run-345f0282-007f-4478-ac21-ba302b12347a-0', usage_metadata={'input_tokens': 128, 'output_tokens': 17, 'total_tokens': 145, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [23]:
model_with_functions.invoke("Give me three songs of Talha Anjum?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Talha Anjum","n":3}', 'name': 'ArtistSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 130, 'total_tokens': 153, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'function_call', 'logprobs': None}, id='run-8211dde7-ba19-414c-be1d-35d01a36e581-0', usage_metadata={'input_tokens': 130, 'output_tokens': 23, 'total_tokens': 153, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [24]:
model_with_functions.invoke("HI?")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 122, 'total_tokens': 132, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-db7ba35d-8d07-48a1-bdf7-4d0b7dc99a46-0', usage_metadata={'input_tokens': 122, 'output_tokens': 10, 'total_tokens': 132, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})